# Fitting a pitch sequence to a sequence of pitch class sets

Sometimes it's musically useful to take a melody and work out what chord changes you can harmonize it with.

This process can be modelled with pitch sequences and pitch class sets.

## The simplest approach

To start, we take our pitch sequence and convert it into a pitch class sequence.

Then, for each pitch class, we will produce a list of pitch class sets that contain that pitch class.

In [8]:
from itertools import product
from more_itertools import powerset
from harmonica.pitch._melody import PitchSeq

modulus = 12

pitch_sequence = PitchSeq([0, 2, 3, 9, 10, 17, 15, 14])
pitch_class_sequence = pitch_sequence.classify(modulus)

pcsets_per_pitch_class = []

for pitch_class in pitch_class_sequence:
    complement = sorted(set(range(modulus)) - set([pitch_class]))
    pcsets = [[pitch_class] + list(extra) for extra in powerset(complement)]
    pcsets_per_pitch_class.append(pcsets)

# pcset_seqs_from_pseq = product(*pcsets_per_pitch_class)

# for pcset_seq in pcset_seqs_from_pseq:
#     print(pcset_seq)

_Holy shit!_ There's no way that can finish computing any time soon. That's (2^11)^8 = 309485009821345068724781056 pcset sequences. 

Granted, I'm definitely not computing this in the most efficient way possible, but... yeah. Just is what it is, I suppose! Can't all be complete.

Moving on...

## Thoughts on chunking

The previous approach is overly naive and simplistic because it assumes one pitch class set per note.

This is basically equivalent to assuming there's one note per chord / key change in the music.

This means that either the melody is moving very slow, or the harmony is moving very quickly.

In practice, there are typically several melody notes played per chord in a progression.

You can represent this by chunking together contiguous notes in the sequence and running the pcset lookup algorithm on them. 

I was originally going to suggest another naive (but more complicated) algorithm that finds _all_ the ways you can group together 
contiguous notes in a melody, and enumerate pcset sequences that fit them. This enumeration would definitely be even BIGGER than the 
previous enumeration.

However, I realized while thinking this through that our original algorithm basically already covers this.

I mean, think about it. Our original algorithm enumerates many pcset sequences where multiple pcsets in a row are identical. 
This yields results that are equivalent to the one's we'd get if we "chunked" together successive pitch classes.

It would, however, still be useful to chunk together successive pitch classes - just probably not useful if we're already enumerating in this one-pcset-per-note manner.

## Tossing junk pitch class sets for musically practical reasons

One thing about our naive algorithm is that a majority of the pitch class sets we consider are not musically useful.

One note, two note, ten note, and eleven note scales just aren't very useful. The same can be said about the chromatic scale, the empty scale, and scales that contain more than 3 semitones in a row. 

The question is, how do we find all the supersets of a pitch class set that fit these properties?

I guess we check for these predicates:

- Cardinality is between 3 and 9
- Interval structure contains no more than 3 (cyclically) successive semitones (1's)

Let's be honest, it'd make the most sense to cache these pcsets.

In [3]:
# get the max successive semitones in a pcset

from typing import Sequence
from harmonica.utility import cycle_diff, rotate


def max_semitones_in_pcset(pcset: Sequence[int], mod: int) -> int:
    diff = cycle_diff(list(pcset), mod, 0)  # max semitones = 3

    if 1 not in diff:
        return 0

    while diff[0] == 1:
        diff = rotate(diff, 1)

    counter = 1
    counting_semitones = False
    max_semitones = 0

    for num in diff:
        if num == 1 and not counting_semitones:
            counting_semitones = True
            counter = 1
        elif num == 1 and counting_semitones:
            counter += 1
        elif num != 1 and counting_semitones:
            if counter > max_semitones:
                max_semitones = counter
            counting_semitones = False
    if counter > max_semitones:
        max_semitones = counter

    return max_semitones

In [4]:
from more_itertools import powerset


def useful_scales(pcset_set: list[list[int]]) -> list[list[int]]:
    scales: list[list[int]] = []
    for pcset in powerset(range(12)):
        if 3 <= len(pcset) <= 8 and max_semitones_in_pcset(pcset, 12) == 1:
            scales.append(list(pcset))
    return scales


print(len(useful_scales([list(x) for x in powerset(range(12))])))

1165


Now let's try that same algorithm from earlier, but with less junk scales.

In [ ]:
from itertools import product
from more_itertools import powerset
from harmonica.pitch._melody import PitchSeq

modulus = 12

pitch_sequence = PitchSeq([0, 2, 3, 9, 10, 17, 15, 14])
pitch_class_sequence = pitch_sequence.classify(modulus)

pcsets_per_pitch_class = []

for pitch_class in pitch_class_sequence:
    complement = sorted(set(range(modulus)) - set([pitch_class]))
    pcsets = useful_scales(
        [[pitch_class] + list(extra) for extra in powerset(complement)]
    )
    pcsets_per_pitch_class.append(pcsets)

# pcset_seqs_from_pseq = product(*pcsets_per_pitch_class)

# for pcset_seq in pcset_seqs_from_pseq:
#     print(pcset_seq)

This is still an absurd amount of harmonizations, even if I ignore scales that have more than 1 successive semitones. 

Perhaps this points to something: there is clearly a rather restricted set of scales that are musically interesting.

I should probably focus on those.

Either way, this is kind of just a junk algorithm, lol!